In [1]:
from scipy.optimize import linprog
import numpy as np

In [2]:
scenario_a = np.random.rand(2)
scenario_b = np.random.rand(2)
p = np.random.rand(1)

# given
print('value(action 0 in scenario a): {}'.format(scenario_a[0]))
print('value(action 1 in scenario a): {}'.format(scenario_a[1]))
print('value(action 0 in scenario b): {}'.format(scenario_b[0]))
print('value(action 1 in scenario b): {}'.format(scenario_b[1]))
print('P(scenario a) : {}'.format(p))
print('P(scenario b): {}'.format(1-p))

# wanted:
# P(action_0) and P(action_1)
#   s.t. p(action_0) + P(action_1) == 1

# target function:
#   P(action 0) * P(scenario a) * value(action 0 in scenario a)
# + P(action 1) * P(scenario a) * value(action 1 in scenario a)
# + P(action 0) * P(scenario b) * value(action 0 in scenario b)
# + P(action 1) * P(scenario b) * value(action 1 in scenario b)

# We use aux. variables to build the opt. problem:
# P(action 0 | scenario a)
# P(action 1 | scenario a)
# P(action 0 | scenario b)
# P(action 1 | scenario b)
# Later we will constraint them so that
# P(action i | scenario x) == P(action i | scenario y) == P(action i)

# Now the variables are:
# 0: P(action_0)
# 1: P(action_1)
# 2: P(action 0 | scenario a)
# 3: P(action 1 | scenario a)
# 4: P(action 0 | scenario b)
# 5: P(action 1 | scenario b)

# target function with aux. variables:
#   P(action 0 | scenario a) * P(scenario a) * value_0a
# + P(action 1 | scenario a) * P(scenario a) * value_1a
# + P(action 0 | scenario b) * P(scenario b) * value_0b
# + P(action 1 | scenario b) * P(scenario b) * value_1b
value = np.concatenate([[0, 0], p * scenario_a, (1 - p) * scenario_b])

# constraints
# 0: P(action 0) + P(action 1) == 1
# 1: P(action 0) == P(action 0 | scenario a)
# 2: P(action 1) == P(action 1 | scenario a)
# 3: P(action 0) == P(action 0 | scenario b)
# 4: P(action 1) == P(action 1 | scenario b)

# Yes, in this simple example we could just get rid of the aux.
# variables, but in the general case we have different constraints
# per scenario.

A = np.array([[1, 1, 0, 0, 0, 0],
              [1, 0, -1, 0, 0, 0],
              [0, 1, 0, -1, 0, 0],
              [1, 0, 0, 0, -1, 0],
              [0, 1, 0, 0, 0, -1]], dtype=float)
b = np.array([1, 0, 0, 0, 0], dtype=float)

print('c: {}\nA:\n{}\nb: {}'.format(value, A, b))

cost = -value
linprog(c=cost, A_eq=A, b_eq=b)

value(action 0 in scenario a): 0.21516681447531516
value(action 1 in scenario a): 0.8478616645301282
value(action 0 in scenario b): 0.022473685583886627
value(action 1 in scenario b): 0.3382141201984563
P(scenario a) : [0.87514193]
P(scenario b): [0.12485807]
c: [0.         0.         0.1883015  0.74199929 0.00280602 0.04222876]
A:
[[ 1.  1.  0.  0.  0.  0.]
 [ 1.  0. -1.  0.  0.  0.]
 [ 0.  1.  0. -1.  0.  0.]
 [ 1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.]]
b: [1. 0. 0. 0. 0.]


     con: array([-2.16227036e-12, -2.60529581e-23, -2.22044605e-16, -1.73683695e-23, -2.22044605e-16])
     fun: -0.7842280558419344
 message: 'Optimization terminated successfully.'
     nit: 4
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([3.79552235e-12, 1.00000000e+00, 3.79552235e-12, 1.00000000e+00, 3.79552235e-12, 1.00000000e+00])